# QuTiP example: Groundstate of an ultra-strong coupled atom-cavity system

This notebook is based on https://nbviewer.jupyter.org/github/qutip/qutip-notebooks/blob/master/examples/ultrastrong-coupling-groundstate.ipynb .


In [21]:
%matplotlib inline
import matplotlib.pyplot as plt
from qutip import *
import numpy as np
import time
from mpl_toolkits.mplot3d import Axes3D

In [22]:
def compute(N, wc, wa, glist, use_rwa):

    # Pre-compute operators for the hamiltonian
    a  = tensor(destroy(N), qeye(2))
    sm = tensor(qeye(N), destroy(2))
    nc = a.dag() * a
    na = sm.dag() * sm
        
    idx = 0
    na_expt = np.zeros(shape(glist))
    nc_expt = np.zeros(shape(glist))
    for g in glist:

        # recalculate the hamiltonian for each value of g
        if use_rwa: 
            H = wc * nc + wa * na + g * (a.dag() * sm + a * sm.dag())
        else:
            H = wc * nc + wa * na + g * (a.dag() + a) * (sm + sm.dag())

        # find the groundstate of the composite system
        evals, ekets = H.eigenstates()
        psi_gnd = ekets[0]
        na_expt[idx] = expect(na, psi_gnd)
        nc_expt[idx] = expect(nc, psi_gnd)

        idx += 1

    return nc_expt, na_expt, ket2dm(psi_gnd)

In [23]:

from ipywidgets import interact

@interact(wc_per_pi=(0, 2.0, 0.5), wa_per_pi=(0, 2.0, 0.5), N=(10, 30, 1), use_rwa=False )
def calculate(wc_per_pi, wa_per_pi, N = 20, use_rwa = False):
    wc = wc_per_pi * 2 * np.pi   # cavity frequency
    wa = wa_per_pi * 2 * np.pi   # atom frequency
    #N = 20              # number of cavity fock states
    #use_rwa = False     # Set to True to see that non-RWA is necessary in this regime
    
    glist = np.linspace(0, 2.5, 50) * 2 * np.pi
    start_time = time.time()
    nc, na, rhoss_final = compute(N, wc, wa, glist, use_rwa)
    print('time elapsed = ' + str(time.time() - start_time)) 
    
    fig, ax = plt.subplots(figsize=(8,6))

    ax.plot(glist/(2*np.pi), nc)
    ax.plot(glist/(2*np.pi), na)
    ax.legend(("Cavity", "Atom excited state"))
    ax.set_xlabel('g - coupling strength')
    ax.set_ylabel('Occupation probability')
    ax.set_title('# photons in the groundstate');
    


interactive(children=(FloatSlider(value=1.0, description='wc_per_pi', max=2.0, step=0.5), FloatSlider(value=1.…